In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd 
import numpy as np

In [3]:
df = pd.read_csv('./data/preprocessed_train_test_data.csv')

In [4]:
df.head()

,date,store,item,sales,id,month,weekday,year,day,weekofyear,...,store_item_sales_mean,store_item_month_sales_sum,store_item_day_sales_sum,store_item_day_sales_median,store_item_weekday_sales_sum,store_item_weekday_sales_median,store_item_dayofyear_sales_sum,store_item_dayofyear_sales_median,store_item_weekofyear_sales_sum,store_item_weekofyear_sales_median
0,2013-01-01,1,1,13.0,NaN,1,1,2013,1,1,...,19.971522,2125.0,1195.0,19.0,4742.0,18.0,73.0,13.0,475.0,13.5
1,2013-01-02,1,1,11.0,NaN,1,2,2013,2,1,...,19.971522,2125.0,1123.0,19.0,4905.0,19.0,72.0,14.0,475.0,13.5
2,2013-01-03,1,1,14.0,NaN,1,3,2013,3,1,...,19.971522,2125.0,1252.0,20.0,5077.0,19.0,63.0,12.0,475.0,13.5
3,2013-01-04,1,1,13.0,NaN,1,4,2013,4,1,...,19.971522,2125.0,1176.0,19.5,5485.0,21.0,75.0,14.0,475.0,13.5
4,2013-01-05,1,1,10.0,NaN,1,5,2013,5,1,...,19.971522,2125.0,1219.0,20.0,5996.0,22.0,67.0,14.0,475.0,13.5


In [5]:
train_data = df.loc[~df['sales'].isna()]
print("train.csv",train_data.shape)
test_data = df.loc[df['sales'].isna()]
print("test.csv",test_data.shape)

train.csv (913000, 85)
test.csv (45000, 85)


In [6]:
train_data.columns

Index(['date', 'store', 'item', 'sales', 'id', 'month', 'weekday', 'year',
       'day', 'weekofyear', 'dayofyear', 'isHoliday', 'isWeekend', 'isSummer',
       'isWinter', 'isAutumn', 'isSpring', 'month_sum', 'month_median',
       'month_mean', 'day_sum', 'day_median', 'day_mean', 'weekofyear_sum',
       'weekofyear_median', 'weekofyear_mean', 'dayofyear_sum',
       'dayofyear_median', 'dayofyear_mean', 'weekday_sum', 'weekday_median',
       'weekday_mean', 'isWeekend_sum', 'isWeekend_median', 'isWeekend_mean',
       'isHoliday_sum', 'isHoliday_median', 'isHoliday_mean', 'isSummer_sum',
       'isSummer_median', 'isSummer_mean', 'isAutumn_sum', 'isAutumn_median',
       'isAutumn_mean', 'isWinter_sum', 'isWinter_median', 'isWinter_mean',
       'store_sales_sum', 'store_sales_median', 'store_sales_mean',
       'store_month_sales_sum', 'store_month_sales_median',
       'store_day_sales_sum', 'store_day_sales_median',
       'store_weekday_sales_sum', 'store_weekday_sales_median'

In [166]:
features = train_data[['store', 
                       'item', 
                       'day', 'month', 'year', 
                       'weekofyear', 'dayofyear', 'weekday',
                      'weekofyear_median', 'dayofyear_median', 
                      'store_sales_median', 'item_sales_median', 
                      'store_item_sales_median', 
                      'store_item_weekofyear_sales_median']] #train_data[['store', 'item', 'month', 'day', 'year', 'weekofyear', 'dayofyear']]
#train_data.drop(['date', 'sales', 'id'], axis=1)
targets = train_data['sales']


In [167]:
features.shape

(913000, 14)

In [168]:
targets.head()

0    13.0
1    11.0
2    14.0
3    13.0
4    10.0
Name: sales, dtype: float64

In [169]:
import keras.backend as K

def custom_smape(x, x_):
    return K.mean(2*K.abs(x-x_)/(K.abs(x)+K.abs(x_)))*100


In [170]:
train_X, test_X, train_y, test_y = train_test_split(features, targets)

In [171]:
from keras.layers import Dense, Dropout, Flatten, SpatialDropout1D
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adadelta, Adam

In [179]:
# define model 
model = Sequential()
model.add(Dense(256, input_dim=len(features.columns)))
model.add(Dense(256, activation=tf.nn.relu))
model.add(Dropout(0.25))
model.add(Dense(256, activation=tf.nn.relu))
model.add(Dropout(0.25))
model.add(Dense(256, activation=tf.nn.relu))
model.add(Dense(256, activation=tf.nn.relu))

model.add(Dropout(0.25))

model.add(Dense(256, activation=tf.nn.elu))
model.add(Dense(256, activation=tf.nn.elu))

model.add(Dropout(0.25))
model.add(Dense(256, activation=tf.nn.elu))

model.add(Dense(1, activation=tf.nn.relu))

In [180]:
model.compile(optimizer=Adadelta(lr=0.03), 
                          loss=custom_smape,
                          metrics=['mean_squared_logarithmic_error']
                            )

In [182]:
from keras.callbacks import TensorBoard
TENSORBOARD_MODEL = 'Initial_Model'
tensorboard = TensorBoard(log_dir=f'./logs/{TENSORBOARD_MODEL}')


In [184]:
epochs = 5
batchsize = 64
model.fit(train_X.values, train_y.values, 
          batch_size=batchsize, 
          epochs=epochs, 
          shuffle=True,
          validation_data=[test_X, test_y], 
          callbacks=[tensorboard]
         )

Train on 684750 samples, validate on 228250 samples
Epoch 1/5
684750/684750 [==============================] - 96s 140us/step - loss: 16.4133 - mean_squared_logarithmic_error: 0.0426 - val_loss: 57.9005 - val_mean_squared_logarithmic_error: 0.3983
Epoch 2/5
684750/684750 [==============================] - 95s 139us/step - loss: 16.3464 - mean_squared_logarithmic_error: 0.0422 - val_loss: 46.1797 - val_mean_squared_logarithmic_error: 0.2629
Epoch 3/5
684750/684750 [==============================] - 96s 140us/step - loss: 16.2704 - mean_squared_logarithmic_error: 0.0419 - val_loss: 47.6221 - val_mean_squared_logarithmic_error: 0.2805
Epoch 4/5
684750/684750 [==============================] - 96s 140us/step - loss: 16.2289 - mean_squared_logarithmic_error: 0.0416 - val_loss: 54.2806 - val_mean_squared_logarithmic_error: 0.3581
Epoch 5/5
684750/684750 [==============================] - 96s 141us/step - loss: 16.1855 - mean_squared_logarithmic_error: 0.0414 - val_loss: 49.3159 - val_mean_sq

In [175]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_133 (Dense)            (None, 256)               3840      
_________________________________________________________________
dense_134 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_135 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_136 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_137 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_24 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_138 (Dense)            (None, 256)               65792     
__________

In [176]:
import time
smape_score = model.evaluate(test_X, test_y)[0]
PROJECT_NAME = 'store_item_demand_forecasting'
MODEL_NAME = f'{PROJECT_NAME}_{smape_score}_{time.time()}'
print(MODEL_NAME)

228250/228250 [==============================] - 9s 38us/step
store_item_demand_forecasting_32.23917347192503_1535317488.2398238


In [158]:
model.save(f'./models/{MODEL_NAME}')

# predictions

In [159]:
test_data_featues = test_data[features.columns]
#test_data.drop(['date', 'id', 'sales'],axis=1)

predictions = model.predict(test_data_featues)

print(predictions)

[[12.416055]
 [12.413456]
 [12.470456]
 ...
 [59.774853]
 [65.70971 ]
 [70.60816 ]]


In [162]:
submission = pd.read_csv('./data/sample_submission.csv.zip')
submission['sales'] = predictions

In [163]:
submission.head()

,id,sales
0,0,12.416055
1,1,12.413456
2,2,12.470456
3,3,12.806584
4,4,12.872875


In [164]:
submission.to_csv('./data/submission.csv', index=False)